In [1]:
import numpy as np
import pandas as pd
import os
from cdt.causality.graph import SAM
import networkx as nx


import matplotlib.pyplot as plt
import seaborn as sns
from tools.reducer import*
from tools.super_learner import*
from tools.TLP import TLP
from scipy.stats import t


from tools.auto_IF import *

from tools.bd_SMDAG import *
from tools.utils import *

Detecting 1 CUDA device(s).


## Causal Pipeline for Psychologists and Social Scientists

This notebook accompanies a paper presenting a causal research pipeline for psychologists and social scientists. The purpose of the notebook is not to detail every possible in a typical research project, but to demonstrate the key steps necessary along the way. It is important to remember, for instance, that deriving a theory about which we are confident can take some back-and-forth, and the output of the causal discovery algorithm is very unlikely to be sufficient on its own. Therefore, in practice, apply the steps in this notebook with caution and dilligence. 


NOTE: Be careful with variable names. We use 'U' as an indicator of unobserved confounders (e.g. 'U1'), thus if variables contain this symbol you may get errors.

### 1. Data Preparation

Here we import and tidy up the data. We also normalize it (regardless of whether the variables are discrete or continuous) because some causal discovery algorithms incorportating score based objectives have been shown to be highly sensitive to variance (Reisach et al. 2021: https://arxiv.org/abs/2102.13647)


In [2]:
fn = 'wide_format_cat_encoded_EPFL.csv'

df = pd.read_csv(fn)
df = df.drop(['RID', 'SID', 'CID', 'RGEND', 'SGEND', 'GENRELNS'], axis=1) # remove uncoded variables
cols = df.columns
print(cols)
print(df.shape)
df.dropna(inplace=True)
print(df.shape)

rename_cols = ['Cohab_Len', 'Age_R', 'Advrs_R', 'Distress_R', 'Support_R', 'Rel_Sat_R', 'R_DCI_S', 'R_DCI_R', 'DCI_Dyd_R', 
               'Dep_R', 'Age_S', 'Advrs_S', 'Distress_S', 'Support_S', 'Rel_Sat_S', 'S_DCI_S', 'S_DCI_R', 'DCI_Dyd_S',
                'Dep_S', 'Rel_Type', 'Gender_R', 'Gender_S']
df.columns = rename_cols


# reorder columns
col_order = ['Age_R', 'Age_S', 'Gender_R', 'Gender_S', 'Rel_Type', 'Advrs_R', 'Advrs_S', 'Distress_R', 'Support_R',
            'Distress_S', 'Support_S', 'R_DCI_S', 'R_DCI_R', 'DCI_Dyd_R', 'S_DCI_S', 'S_DCI_R', 'DCI_Dyd_S', 'Cohab_Len', 
            'Rel_Sat_R', 'Rel_Sat_S', 'Dep_R', 'Dep_S']
df = df[col_order]

cols = df.columns

print(cols, len(cols), len(col_order),)

# standardize the data for causal discovery
df_cd = (df-df.mean()) / df.std()
cols

Index(['YRSLIV', 'RAGE', 'RTOTADVRS', 'Rdistress_intensity', 'Rsupport',
       'Rsatisfaction', 'RDCI_s_response_r', 'RDCI_r_response_s',
       'RDCI_dyadic', 'RDEPSYM', 'SAGE', 'STOTADVRS', 'Sdistress_intensity',
       'Ssupport', 'Ssatisfaction', 'SDCI_s_response_r', 'SDCI_r_response_s',
       'SDCI_dyadic', 'SDEPSYM', 'GENRELNS_coded', 'RGEND_coded',
       'SGEND_coded'],
      dtype='object')
(419, 22)
(403, 22)
Index(['Age_R', 'Age_S', 'Gender_R', 'Gender_S', 'Rel_Type', 'Advrs_R',
       'Advrs_S', 'Distress_R', 'Support_R', 'Distress_S', 'Support_S',
       'R_DCI_S', 'R_DCI_R', 'DCI_Dyd_R', 'S_DCI_S', 'S_DCI_R', 'DCI_Dyd_S',
       'Cohab_Len', 'Rel_Sat_R', 'Rel_Sat_S', 'Dep_R', 'Dep_S'],
      dtype='object') 22 22


Index(['Age_R', 'Age_S', 'Gender_R', 'Gender_S', 'Rel_Type', 'Advrs_R',
       'Advrs_S', 'Distress_R', 'Support_R', 'Distress_S', 'Support_S',
       'R_DCI_S', 'R_DCI_R', 'DCI_Dyd_R', 'S_DCI_S', 'S_DCI_R', 'DCI_Dyd_S',
       'Cohab_Len', 'Rel_Sat_R', 'Rel_Sat_S', 'Dep_R', 'Dep_S'],
      dtype='object')

### 2. Create a skeleton which constrains the causal discovery

Here we use our initial theory to constrain the possible causal structure (e.g. no links backwards in time).

In [3]:
# nothing can cause:
no_causes = [0,1,2,3,4,5,6]

# constrain causal links
skeleton = 1- np.eye((len(cols)),dtype=np.float32)
skeleton[:, no_causes] = 0

# specify more specific causal link (or rather, the absence thereof)
group_cause_A = np.array([11,12,13,14,15,16,17])  
group_effect_A = np.array([7,8,9,10])  # group cause A cannot affect group effect A

group_cause_B = np.array([18, 19, 20, 21])
group_effect_B = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])  # group cause B cannot affect group effect B


for cause in group_cause_A:
    for effect in group_effect_A:
        skeleton[cause, effect] = 0
for cause in group_cause_B:
    for effect in group_effect_B:
        skeleton[cause, effect] = 0
        
print(skeleton, skeleton.shape) 

[[0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

### 3. Run causal disc.

Here we use SAM (Kalainathan et al. 2020). We take the result averaged over 50 initializations.

An alternative approach is given in this markdown cell, too. Note that we have not had good experiences with mutual information (https://github.com/syanga/pycit/blob/master/pycit/estimators/mixed_cmi.py) or the GCIT (https://github.com/alexisbellot/GCIT) approaches. But they are options.

```python
from pc_alg2 import *

alpha = 0.01  # false positive rate
knn = 500  # number of nearest neighbours for MI based tests (in general recommend knn /approx (N/10))

# Using the modified PC alg originally from pgmpy which takes the additional MI based ci tests
# note that mi based tests will take quite some time to run!
''' ci_test = {'pearsonr', 'gcit', 'chi_square', 'mixed_mi', 'cont_mi', 'independence_match'}
'''

# gcit uses https://github.com/alexisbellot/GCIT/blob/master/Tutorial.ipynb
c = PC_adapted(df_cd)
model = c.estimate(ci_test="gcit", return_type="cpdag", significance_level=alpha, knn=knn)
nx.draw(model, with_labels=True, node_color='white', edge_color='k',
        node_size=500, font_size=25, arrowsize=20, )
```

In [ ]:
# Using SAM

'''lr (float) – Learning rate of the generators
dlr (float) – Learning rate of the discriminator
mixed_data (bool) – Experimental – Enable for mixed-type datasets
lambda1 (float) – L0 penalization coefficient on the causal filters
lambda2 (float) – L2 penalization coefficient on the weights of the neural network
nh (int) – Number of hidden units in the generators’ hidden layers (regularized with lambda2)
dnh (int) – Number of hidden units in the discriminator’s hidden layers
train_epochs (int) – Number of training epochs
test_epochs (int) – Number of test epochs (saving and averaging the causal filters)
batch_size (int) – Size of the batches to be fed to the SAM model Defaults to full-batch
losstype (str) – type of the loss to be used (either ‘fgan’ (default), ‘gan’ or ‘mse’)
dagloss (bool) – Activate the DAG with No-TEARS constraint
dagstart (float) – Controls when the DAG constraint is to be introduced in the training (float ranging from 0 to 1, 0 denotes the start of the training and 1 the end)
dagpenalisation (float) – Initial value of the DAG constraint
dagpenalisation_increase (float) – Increase incrementally at each epoch the coefficient of the constraint
functional_complexity (str) – Type of functional complexity penalization (choose between ‘l2_norm’ and ‘n_hidden_units’)
hlayers (int) – Defines the number of hidden layers in the generators
dhlayers (int) – Defines the number of hidden layers in the discriminator
sampling_type (str) – Type of sampling used in the structural gates of the model (choose between ‘sigmoid’, ‘sigmoid_proba’ and ‘gumble_proba’)
linear (bool) – If true, all generators are set to be linear generators
nruns (int) – Number of runs to be made for causal estimation Recommended: >=32 for optimal performance
njobs (int) – Numbers of jobs to be run in Parallel Recommended: 1 if no GPU available, 2*number of GPUs else
gpus (int) – Number of available GPUs for the algorithm
verbose (bool) – verbose mode
'''

lrs = [0.01]
daglosses = [True]
dagpenalizations = [0.06]
num_runs = 50



'''skeleton (numpy.ndarray) – 
A priori knowledge about the causal relationships as an adjacency matrix. 
Can be fed either directed or undirected links.
'''


for lr in lrs:
    for dagloss in daglosses:
        for dagpenalization in dagpenalizations:
            print(dagpenalization)
            
            settings = 'lr_' + str(lr) + '_dagloss_' + str(dagloss) + '_dagpen_' + str(dagpenalization) + '_numruns_' + str(num_runs)
            
            obj = SAM(lr=lr, dlr=0.001, mixed_data=True, lambda1=10, lambda2=0.001, nh=20,
                       dnh=200, train_epochs=3000, test_epochs=800, batch_size=- 1, losstype='fgan',
                       dagloss=dagloss, dagstart=0.5, dagpenalization=0, dagpenalization_increase=dagpenalization, 
                        hlayers=2, dhlayers=2, sampling_type='sigmoidproba',
                       linear=False, nruns=num_runs, njobs=1, gpus=1, verbose=None)

            output = obj.predict(df_cd, graph=skeleton.T) 
            
            nx.write_gml(output, "graphs/SAM_solution_{}.gml".format(settings))

            
       
    

0.06


 72%|███████▏  | 2723/3800 [00:42<00:18, 57.42it/s, disc=-.13, gen=-.0615, regul_loss=2.04, tot=-20]     

### 4. Threshold the discovered structure.

SAM gives us an adjacency matrix with continuous 'confidences' between 0 and 1. Thus, we need to decide on a threshold. Looking at the distribution of confidences can give us some idea of where to threshold. 

In [ ]:

title ='S'
graph_name = 'SAM_solution_lr_0.01_dagloss_True_dagpen_0.05_numruns_50.gml'
graph_name_short = '.'.join(graph_name.split('.')[:-1])

output = nx.read_gml("graphs/" + graph_name, destringizer=int)

threshold = 0.5
G_thresh = threshold_graph(output, threshold=threshold)
plot_subbgraph(G=G_thresh, variables=cols, subgraph_name=title+graph_name_short + '_all_{}'.format(threshold), size=(16,12), plot_adj=True, threshold=threshold)

### 5. Remove any cycles in the graph.

We plan to use the DAG framework to identify and estimate the causal effect of interest. We therefore need the graph to be acylclic. In practice, some time may be required to remove these links or integrate them into the theory and use a different analytical method. For the purposes of demonstration, we just remove the links as they appear.

In [ ]:
glen = 1
while glen > 0:
    cycle_list = list(nx.simple_cycles(G_thresh))
    print(cycle_list)
    glen = len(cycle_list)
    if glen == 0:
        break
    else:
        G_thresh.remove_edge(cycle_list[0][0], cycle_list[0][1])
        

... As it turns out, there were no cycles anyway.



### 6. Add unobserved confounders with edge plausibility scores
Causal discovery algorithms are not good at inferring unobserved confounders with observational data. Therefore, to improve the plausibility of our discovered graph, we should be conservative and incorporate some unobserved confounders.

In particular, we include some confounders between dyadic variables (like relationships satisfaction for each partner).

In [ ]:
# assign weights of 1 to 'observed' edges
for u,v,d in G_thresh.edges(data=True):
    d['weight'] = 1.0

# introduced confounders and probabilities betwen 0.5 and 1 
# (probs < 0.5 are less than likely to exist, so are omitted)
G_thresh.add_edge('U1','Dep_S', weight=0.6)
G_thresh.add_edge('U1','Dep_R', weight=0.6)
G_thresh.add_edge('U2','Rel_Sat_S', weight=0.7)
G_thresh.add_edge('U2','Rel_Sat_R', weight=0.7)
G_thresh.add_edge('U3','Support_R', weight=0.6)
G_thresh.add_edge('U3','Support_S', weight=0.6)
G_thresh.add_edge('U4','Dep_R', weight=0.55)
G_thresh.add_edge('U4','Distress_R', weight=0.55)
G_thresh.add_edge('U4','Dep_S', weight=0.55)
G_thresh.add_edge('U4','Distress_S', weight=0.55)
G_thresh.add_edge('U5','S_DCI_S', weight=0.55)
G_thresh.add_edge('U5','S_DCI_R', weight=0.55)
G_thresh.add_edge('U6','DCI_Dyd_R', weight=0.55)
G_thresh.add_edge('U6','DCI_Dyd_S', weight=0.55)


### 7.  Reduce the graph 

The graph is evidently quite large, and yet we may only be interested in estimating a very specific effect. This is not a bad thing - we should have a large graph to ensure we get (for example) the variables necessary to at least form a Markov Blanket around the effect(s) we care about. Once we have our graph and our research question, however, there are likely opportunities to reduce the graph to the key variable for estimation (backdoor adjustment variables, the cause and outcomes of interest, and any precision variables).

The causal pipeline package has a number of ways to do this. But given that we have edge probabilities, we should start with the backdoor semi markovian reduction algorithms in bd_SMGDAG.

In [ ]:
outcome = 'Dep_R'
cause = 'Distress_R'

# TEST BACKDOOR ID already
check, bd_set = check_bd_id(graph=G_thresh, X=cause, Y=outcome)

if check:
    print('Is identifiable with backdoor set:', bd_set)
else:
    print('Backdoor criterion not fulfilled, finding confounders to remove...')
    # RUNNING BRUTE FORCE ALGO TO FIND SOLUTION AND COST
    E, best_cost = bd_brute(graph=G_thresh, X=cause, Y=outcome)

    if best_cost != np.inf:
        print('Identifiable with the removal of:', E, ' at a cost of:', best_cost)
    else:
        print('No solution.')

    if best_cost != 0:
        # FINDING PLAUSIBILITY RATIO OF SOLUTION TO ORIGINAL GRAPH
        before_logsum, after_logsum, cutset_invlogsum = get_probs(E, G_thresh)
        ratio = np.exp((after_logsum + cutset_invlogsum)) / np.exp(before_logsum)
        print('Plausibility ratio:', ratio)
    else:
        ratio = 1.0

    # REMOVE NODES FROM GRAPH AND TEST BACKDOOR ID AGAIN, PROVIDING SUFFICIENT ADJUSTMENT SET
    for node in E:
        G_thresh.remove_node(node)

    check, bd_set = check_bd_id(graph=G_thresh, X=cause, Y=outcome)

    if check:
        print('Is identifiable with backdoor set:', bd_set)
    else:
        print('Backdoor criterion not fulfilled.')



The results above tell us that U4 was stopping us from achieving backdoor identification, and that after its removal, our graph is given a before-after plausibility ratio of 0.8. One can interpret this as - our new graph is 80% as plausible as the original graph, assuming the original was correct.

### 8. Remove remaining unobserved confounders
Now that we have removed unobserved confounders which affect our backdoor adjustment, we can also remove any which remain - we cannot do anything with them because they are unobserved!


In [ ]:
nodes = list(G_thresh.nodes())

for node in nodes:
    if 'U' in node:
        G_thresh.remove_node(node)

### 9. Reduce graph and identify backdoor set and precision variables
Technically, we already have what we need from step  7 above. But sometimes we can improve estimation if we include precision variables. These are variables which do not help us identify the effect we care about, but which may help improve the precision of our estimate in finite samples. The next step identifies the backdoor variables and precision variables, and it also reduces the graph according to 'projection' - e.g. mediated paths are combined, and the precision variables are removed from the graph. The graph reduction step isn't necessary for our subsequent estimation, but it can be used to provide a minimal graph for purposes of linear SEM modeling.

In [ ]:
rd, confs, precs = reducer(G_thresh, [cause], [outcome],  remove_precision=False, project_causes=True, project_confs=True)

print('Number of edges before reduction: ', len(list(G_thresh.edges())))
print('Number of edges after reduction: ', len(list(rd.edges())))

# plot the reduced graph
plot_subbgraph(G=rd, variables=cols, subgraph_name=title+graph_name_short + '_reduced_{}'.format(threshold), size=(16,12), plot_adj=True)

### (alternative) Identify the Effect (again... so not  really necessary) 

We already know the effect is identifiable under the backdoor criterion, but we can demonstrate the use of an alternative approach to general identification, in case it is useful. 

For this we use the causaleffect package https://github.com/pedemonte96/causaleffect  https://arxiv.org/abs/2107.04632

N.B. The function only accepts graph variable/node names without spaces. e.g. 'Dep R' should be 'Dep_R'. verbose=False is recommended.

In [ ]:

s = check_id(rd, cause=cause, effect=outcome, verbose=False)

# if one needs to view the formula for identification
# display(Math(s))

### 10. Targeted Learning and Inference

Let us recap what we have done so far

- causal discovery
- added unobserved confounders with edge plausibilities
- identified the most plausible subgraph that facilitates backdoor adjustment
- reduce this subgraph futher to contain the nodes and edges necessary to estimate a target effect
- establish a list of backdoor adjustment variables and precision variables

Now we are ready to do some estimation. The process can be broken down as follows:

1. Specify the outcome min and max - we will transform the outcome into the range 0 to 1, so need the max and mins of the scale! 
2. Specify the SuperLearner dictionaries for the treatment(G) and outcome (Q) models  See Phillips et al. (2022) for some recommendations https://arxiv.org/abs/2204.06139
3. Check list of confounders and precision variables. It so happens that there are no confounders for this choice of treatment and outcome. This is not a problem for tareted learning - we simply compute the marginal probabilities of belonging to a certain treatment group, and the G superlearner is not used.
4.  Specify whether the outcome is continuous of categorical (in our case it is continuous)
5. Make sure the treatment variable has the reference group as 0. For example, in our data, Distress_R ranges from 1 to 5, so if we want group 1 to be the reference, we subtract one from all datapoints.  Also specify the group comparisons required for the chosen treatment.
6. Set the number of folds in k-fold. See Phillips et al. (2022) for some recommendations https://arxiv.org/abs/2204.06139
7. Initialise a TLP targeted learning class and...
8. run targeted learning

In [ ]:
# step 1 - we need the scale min and max bounds so that we can scale the outcome between 0 and 1
# in practice set this to the actual scale range (rather than the empirical range)
outcome_scale_ranges = [df[outcome].min(), df[outcome].max()]  
outcome_min, outcome_max = outcome_scale_ranges[0], outcome_scale_ranges[1]

# step 2 - specify the superlearner candidate learners
est_dict_Q = ['Elastic', 'BR', 'SV', 'LR', 'RF', 'MLP', 'AB', 'poly']
est_dict_G = ['LR', 'NB', 'MLP','SV', 'poly', 'RF','AB']

# step 3
i = 0
print('outcome: ', outcome, '. cause: ', cause, '. Confounders:', confs, '. Precisions:', precs, '\n')

# step 4
outcome_type = 'reg'  # 'reg' or 'cls'

# step 5
df.Distress_R = df.Distress_R - 1  # don't do this multiple times!
group_comparisons =[[2,0], [4,0]]  # comparison in list format with 'group B [vs] reference_group'

In [ ]:
# step 6
k = 8  # number of folds for SL training

# step 7
tlp = TLP(df, cause=cause, outcome=outcome, confs=list(confs),
          precs=list(precs), outcome_type=outcome_type, Q_learners=est_dict_Q, G_learners=est_dict_G,
         outcome_upper_bound=outcome_max, outcome_lower_bound=outcome_min, seed=0)

# step 8 
 # fit SuperLearners. NB standardized_outcome is just for the fitting (this is totally separate to the targeted learning scaling)
all_preds_Q, gts_Q, all_preds_G, gts_G = tlp.fit(k=k, standardized_outcome=False, calibrationQ=True, calibrationG=False)

# 'do' targeted learning
pre_update_effects, post_update_effects, ses, ps = tlp.target_multigroup(group_comparisons=group_comparisons)

# 'do' targeted learning with double robust inference
# pre_update_effects_dr, post_update_effects_dr, ses_dr, ps_dr = tlp.dr_target_multigroup(group_comparisons=group_comparisons)


naive_effects = {}
naive_ses = {}
naive_ps = {}
for group_comparison in group_comparisons:
    group_a = group_comparison[0]
    group_ref = group_comparison[1]

    data_a = (df[df[cause] == group_a][outcome] - outcome_min) / (outcome_max - outcome_min)
    data_ref = (df[df[cause] == group_ref][outcome] - outcome_min) / (outcome_max - outcome_min)
    na = len(data_a)
    nref = len(df)
    naive = data_a.mean() - data_ref.mean()
    data_a_se = np.std(data_a, ddof=1) / (np.sqrt(na))
    data_ref_se = np.std(data_ref, ddof=1) / (np.sqrt(nref))
    diff_se = np.sqrt(data_a_se**2 + data_ref_se**2)

    upper, lower = naive + 1.96*diff_se, naive - 1.96*diff_se

    p = 2 * (1 - t.cdf(np.abs(naive) /diff_se, na+nref))

    naive_effects[str(group_comparison)] = naive
    naive_ses[str(group_comparison)] = (diff_se, upper, lower)
    naive_ps[str(group_comparison)] = p

### 11. Plot the results / perform inference

We plot the results below. We can see that the targeting step has increased the precision of the estimates compared with the naive group differences. In this example, there were no confounders for our choice of cause and effect. But one could set the list of confounders to be empty (if it isn't) and re-run it, to get an estimation for how much the confounders are affecting the estimate under deliberate structural misspecification. With this one can create a sensitivity analysis by comparing the results for different amounts of structural misspecificatoin. One can also discount the validity of the results according to the plausibility ratio we derived when establishing the backdoor identifiable subgraph.

In [ ]:
plt.rcParams.update({'font.size': 22})
plt.figure(figsize=(12,8))


est_2_0 = post_update_effects['[2, 0]']
err_2_0 = 1.96 * ses['[2, 0]'][0]

est_2_0_pre = pre_update_effects['[2, 0]']

est_2_0_naive = naive_effects['[2, 0]']
err_2_0_naive = 1.96 * naive_ses['[2, 0]'][0]

est_4_0 = post_update_effects['[4, 0]']
err_4_0 = 1.96 * ses['[4, 0]'][0]

est_4_0_pre = pre_update_effects['[4, 0]']

est_4_0_naive = naive_effects['[4, 0]']
err_4_0_naive = 1.96 * naive_ses['[4, 0]'][0]

plt.errorbar(x=[outcome + '  2vs0 naive'], y=est_2_0_naive, yerr=err_2_0_naive, capsize=5, fmt='x', color='teal', label='naive')
plt.errorbar(x=[outcome + ' 2vs0 pre'], y=est_2_0_pre, yerr=0, capsize=10, fmt='o', color='r', label='pre-targeted')
plt.errorbar(x=[outcome + ' 2vs0 targeted'], y=est_2_0, yerr=err_2_0, capsize=10, fmt='o', color='k', label='targeted')

plt.errorbar(x=[outcome + ' 4vs0 naive'], y=est_4_0_naive, yerr=err_4_0_naive, capsize=5, fmt='x', color='teal')
plt.errorbar(x=[outcome + ' 4vs0 pre'], y=est_4_0_pre, yerr=0, capsize=10, fmt='o', color='r')
plt.errorbar(x=[outcome + ' 4vs0 targeted'], y=est_4_0, yerr=err_4_0, capsize=10, fmt='o', color='k')


plt.xlabel('Outcome')   
plt.axhline(y = 0, color = 'k', linestyle = '-') 
plt.grid()
plt.tight_layout()
plt.xticks(rotation = 90)
plt.ylim(-.22, .3)
plt.legend()
plt.savefig('tl_results/' + outcome+'_targeted_comparison.png', bbox_inches="tight", dpi=200)
plt.show()

### 12. Sensitivity Analysis

Run an analysis for 'worst case' scenario (no adjustment). In the running example, we actually didn't identify any confounders vai causal discovery anyway, and researchers should really investigate this further before continuing (especially for psychology, where there are confounders galore...). However, for the sake of the illustration, we assume that our precision variables were actually miss-allocated confounders, and so for sensitivity analysis, we remove them all and re-run the analysis to understand the impact. This we quantify as $\delta = 1$, and we can then great a family of results for different multiples of $\delta$ to understand how much confounding would be necessary in practice to change our inference.


In [ ]:
# step 1
k = 8  # number of folds for SL training

# step 2 - specify empty confounders and precision sets
tlp = TLP(df, cause=cause, outcome=outcome, confs=[],
          precs=[], outcome_type=outcome_type, Q_learners=est_dict_Q, G_learners=est_dict_G,
         outcome_upper_bound=outcome_max, outcome_lower_bound=outcome_min, seed=0)

# step 3
 # fit SuperLearners
all_preds_Q, gts_Q, all_preds_G, gts_G = tlp.fit(k=k, standardized_outcome=False, calibrationQ=True, calibrationG=False)

# step 4 'do' targeted learning
pre_update_effects_delta, post_update_effects_delta, ses_delta, ps_delta = tlp.target_multigroup(group_comparisons=group_comparisons)



# step 5 specify contrast of interest
grp = '[4, 0]'

# step 6 specify the multipls / values of delta you want to explore 
multiples = [0.5,1, 1.5, 2, 2.5]


# step 7
# get the difference between our intentionally biased estimate and the one for which we assume delta = 0 
est_biased = post_update_effects_delta[grp]

est_unbiased = post_update_effects[grp]
err_unbiased = 1.96 * ses[grp][0]

diff_est = np.abs(est_biased - est_unbiased)


# step 8 get the plots
# plot negative range of multiples of delta 
plt.rcParams.update({'font.size': 22})
plt.figure(figsize=(12,8))

for multiple in reversed(multiples):
    biased_est_neg = est_unbiased - multiple*diff_est  # get original - amount of bias
    plt.errorbar(x=['-{}'.format(multiple)], y=biased_est_neg, yerr=err_unbiased, capsize=10, fmt='o', color='k', label='pre-targeted')

# plot original estimate (assuming delta=0) 
plt.errorbar(x=['0'], y=est_unbiased, yerr=err_unbiased, capsize=10, fmt='o', color='k', label='pre-targeted')
plt.axhline(y=0.0, color='r', linestyle='--')
plt.axvline(x=5, color='r', linestyle='--')
    
for multiple in multiples:
    biased_est = est_unbiased + multiple*diff_est # get original + amount of bias
    
    plt.errorbar(x=['{}'.format(multiple)], y=biased_est, yerr=err_unbiased, capsize=10, fmt='o', color='k', label='pre-targeted')

plt.ylim(-.22, .7)
plt.xlabel('delta')   

plt.grid()
plt.tight_layout()
plt.savefig('tl_results/' + outcome+'_targeted_comparison_sensitivity_4-0.png', bbox_inches="tight", dpi=200)
plt.show()